In [1]:
import random
import uuid
from collections import Counter

from typed_lists import TypedList, _initialize_global_thread_pool

## Examples

In [2]:
# can set number of threads to use for parallel operations
# this is a global setting and can only be set once
# calling this function again will raise an error

_initialize_global_thread_pool(4)

In [3]:
# examples of creating a typed list 
print(TypedList(range(10)) * TypedList(range(10)))
print(TypedList(["a", "b", "c", "d", "e"]) + TypedList(["f", "g", "h", "i", "j"]))
print(TypedList([1.0, 2.0, 3.0, 4.0, 5.0]) / TypedList([1.0, 2.0, 3.0, 4.0, 5.0]))
print(TypedList([bool(i%2) for i in range(10)]))

IntTypedList([0, 1, 4, 9, 16]...)
StringTypedList (["af", "bg", "ch", "di", "ej"])
FloatTypedList ([1.0, 1.0, 1.0, 1.0, 1.0])
BoolTypedList([false, true, false, true, false]...)


In [4]:
# parallelized sorting
uuids = TypedList([str(uuid.uuid4()) for _ in range(1_000_000)])
uuids.sort()
uuids

StringTypedList(["d5423c84-10f7-4793-911d-11ba755b8c06", "adcbff9e-c3a3-4b81-9d7f-7947bc8757df", "1cab3ab7-88af-4af5-8a49-b52af5940024", "b2656bfe-c6b9-4930-acf1-b05f7269cd58", "bec4d017-b127-4bac-8840-f63af4fce446"]...)

In [5]:
# parallelized counting
ids = TypedList([random.randint(0, 4) for _ in range(1_000_000)])
ids.count_all()

{1: 199592, 2: 199572, 4: 200179, 3: 199932, 0: 200725}

In [6]:
# broadcasting and parallelized operations
a = TypedList(range(1_000_000))
b = TypedList(range(1_000_000))
a + b * 3

IntTypedList([0, 4, 8, 12, 16]...)

In [7]:
# slice access
print(TypedList(range(10))[1:5])

# slice assignement
my_typed_list = TypedList(range(10))
my_typed_list[:4] = TypedList(range(4, 0, -1))
print(my_typed_list)

IntTypedList ([1, 2, 3, 4])
IntTypedList([4, 3, 2, 1, 4]...)


In [8]:
# for bool you can use the &, |, and ^ operators
# and they're parallelized and broadcasted
# Note: __bool__ still corresponds to the length of the list

# fizzbuzz example
fizz = TypedList(range(1, 1_000_000)) % 3 == 0
buzz = TypedList(range(1, 1_000_000)) % 5 == 0
print(f"fizz buzz: {(fizz & buzz).count_all()}")

fizz buzz: {True: 66666, False: 933333}


In [9]:
# comparing typed lists always generates a BoolTypedList 
print(TypedList(range(10)) == 2)
print(TypedList(range(10)) == TypedList(range(10)))

# BoolTypedLists can be collapsed to a single bool w/ any or all
print((TypedList(range(10)) == 2).any())

# BoolTypedLists can be used as a mask
print(
    TypedList(range(10))[TypedList([bool(i%2) for i in range(10)])]
)

BoolTypedList([false, false, true, false, false]...)
BoolTypedList([true, true, true, true, true]...)
True
IntTypedList ([1, 3, 5, 7, 9])


In [10]:
# find the closest point to origin
x1, x2 = (
    TypedList((random.random() for _ in range(1_000))),
    TypedList((random.random() for _ in range(1_000))),
)
y1, y2 = (0, 0)
dists = ((x1 - y1) ** 2 + (x2 - y2) ** 2) ** 0.5
closest = dists.argmin()
x1[closest], x2[closest]

(0.006970171593920438, 0.007404486981432257)

## Performance Example

In [11]:
# count "random" Strs
rust_strs = TypedList([str(random.randint(1, 100)) for _ in range(100)] * 10_000)
py_strs = rust_strs.data

In [12]:
%%timeit
_ = rust_strs.count_all()

11.3 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
_ = Counter(py_strs)

38 ms ± 42.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
# fizzbuzz example
# note: creating the TypedList is a a bit slower, 
# but the operations are faster
candidates = range(1, 10_000_000)
typed_list = TypedList(candidates)
py_list = list(candidates)

In [15]:
%%timeit
fizz = typed_list % 3 == 0
buzz = typed_list % 5 == 0
_ = typed_list[fizz & buzz]

28.6 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
fizzbuzz = [i for i in py_list if  i % 3 == 0 and i % 5 == 0]

332 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
# find closest point to origin
x1, x2 = (
    [random.random() for _ in range(1_000_000)],
    [random.random() for _ in range(1_000_000)],
)
typed_x1, typed_x2 = TypedList(x1), TypedList(x2)
y1, y2 = (0, 0)

In [18]:
%%timeit
dists = ((typed_x1 - y1) ** 2 + (typed_x2 - y2) ** 2)
min_ix = dists.argmin()
_ = x1[min_ix], x2[min_ix]

9.79 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
indices = range(len(x1))
min_ix = min(indices, key = lambda i: (x1[i] - y1) ** 2 + (x2[i] - y2) ** 2)
_ = x1[min_ix], x2[min_ix]

168 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
